# Réalisez un dashboard
## Notebook - Test de l'API  
OpenClassrooms - Parcours Data Scientist - Projet 08  

__Ce notebook teste l'API en ligne :__  
* Sélection d'un exemple dans le jeu de test  
* Interrogation de l'API  
* Affichage de la probabilité de défaut de remboursement
* Affichage de l'accord ou non du crédit
* Affichage des 20 éléments ayant eu le plus d'impact sur la prédiction

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os

In [2]:
# Nombre d'exemples à tester
n_samples = 1

In [3]:
# Récupération des données de test
base_dir = os.getcwd()
df_test_path = os.path.join(base_dir, '..', '..', 'P08 - Dashboard', 'data', 'raw', 'application_test.csv')
df_test = pd.read_csv(df_test_path)

In [4]:
# Récupération du seuil de classification
threshold_path = os.path.join(base_dir, '..', 'data', 'processed', 'best_threshold.txt')
with open(threshold_path, 'r') as threshold_file:
    best_threshold = float(threshold_file.read())

In [5]:
# Sélection aléatoire de n_samples exemples
df_subset = df_test.sample(n_samples)

In [6]:
# Récupération des id clients
sk_id_curr = df_subset['SK_ID_CURR'].astype('int')

In [7]:
# Suppression des id clients du subset
df_subset = df_subset.drop(columns='SK_ID_CURR')

In [8]:
# Remplacement des valeurs manquantes par None (JSON n'accepte pas les NaN)
df_subset =  df_subset.map(lambda x: None if pd.isna(x) else x)

In [9]:
# Conversion au format JSON
data_json = json.dumps({
    "columns": df_subset.columns.tolist(),
    "data": df_subset.values.tolist()}
)

In [10]:
# Envoi de la requête POST à l'API
heroku_url = 'https://failurescore-bc9f53f25e58.herokuapp.com/predict'
local_url = 'http://127.0.0.1:5000/predict'
response = requests.post(
    heroku_url,
    headers={'Content-Type': 'application/json'},
    data=data_json
)

In [11]:
# Vérification du code status
display(response)

<Response [200]>

In [12]:
# DataFrame pour affichage des probabilités et accord ou non de crédit
predictions = pd.DataFrame({'Customer id': sk_id_curr.values.tolist(), 'Failure probability': response.json()['prediction_proba'], 'Credit agreement': response.json()['prediction_class']})
predictions['Credit agreement'] = predictions['Credit agreement'] == 0
predictions['Failure probability'] = round(predictions['Failure probability'], 2)

In [13]:
# Affichage
display(predictions)

,Customer id,Failure probability,Credit agreement
0,435120,0.11,True


In [14]:
# DataFrame pour affichage Feature importance locale
local_fi = pd.DataFrame({'Feature': response.json()['feature_names'], 'Importance': response.json()['feature_importance'][0]})

In [15]:
# Affichage des 20 features ayant eu le plus d'impact sur la prédiction
local_fi.sort_values(by='Importance', key=np.abs, ascending=False).head(20)

,Feature,Importance
30,EXT_SOURCE_1,-0.546744
31,EXT_SOURCE_2,-0.370642
32,EXT_SOURCE_3,-0.333418
145,OCCUPATION_TYPE_Core staff,-0.088035
10,DAYS_EMPLOYED,-0.082119
6,AMT_ANNUITY,-0.071657
22,REGION_RATING_CLIENT_W_CITY,-0.058417
37,COMMONAREA_AVG,-0.057240
13,OWN_CAR_AGE,0.053873
195,ORGANIZATION_TYPE_Kindergarten,-0.052432
